Install the necessary modules

In [ ]:
!pip install -q transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 10.9 MB/s eta 0:00:00


In [ ]:
!pip install datasets
#!pip install -q transformers datasets
#!pip install -q --upgrade transformers
#!pip install -q --upgrade datasets
!pip install -q accelerate

# !pip uninstall -y transformers accelerate
# !pip install transformers accelerate

!pip install -q --upgrade accelerate


## Load dataset

Download a multi-label text classification dataset from the [hub](https://huggingface.co/).



In [ ]:
from datasets import load_dataset

dataset = load_dataset("sem_eval_2018_task_1", "subtask5.english")

The dataset contains 3 splits: one for training, one for validation and one for testing.

In [ ]:
dataset

Example of the training split:
The dataset contains ID, tweet, and labeled emotions.

In [ ]:
example = dataset['train'][0]
example


Creating a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [ ]:
labels = [label for label in dataset['train'].features.keys() if label not in ['ID', 'Tweet']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

## Preprocess data for logistic classification

Since text is not a format that is easy to train a model with, we are transforming the strings into vectors using sklearn's TfidfVectorizer.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

def preprocess_data(dataset, labels):
    X, Y = [], []
    for row in dataset:
        X.append(row['Tweet'])
        Y.append([1 if row[label] else 0 for label in labels])
    return X,Y
    
X_train, y_train = preprocess_data(dataset['train'], labels)
X_val, y_val = preprocess_data(dataset['validation'], labels)
X_test, y_test = preprocess_data(dataset['test'], labels)

# Vectorize the tweets
X_train= vectorizer.fit_transform(X_train)
X_val= vectorizer.transform(X_val)
X_test = vectorizer.transform(X_test)


As you can see below, the tweet and the emotion labels are represented as vectors.

In [ ]:
print(X_train[0])
print(y_train[0])

## Define Logistic Classification model

We need to use multi target classification because each tweet can correspond to multiple emotions. We are going to use sklearn's MultiOutputClassifier with Logistic Regression as the estimator.

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

clf = MultiOutputClassifier(LogisticRegression()).fit(X_train, y_train)


## Test the model
Use the test dataset to calculate the accuracy of the trained Logistic Regression model. 

In [ ]:
# Change the vector prediction to emotions
def vectorToEmotions(y, labels):
    return [id2label[idx] for idx, label in enumerate(y[i]) if label == 1.0]

In [ ]:
y_preds = clf.predict(X_train[:5])
for i in range(len(y_preds)):
    print(vectorToEmotions(y_preds,labels ))
    print(vectorToEmotions(y_train[:5], labels))

['optimism']
['anticipation', 'optimism', 'trust']
['joy', 'optimism']
['joy', 'love', 'optimism']
['disgust', 'joy']
['anger', 'disgust', 'joy', 'optimism']
['joy', 'optimism']
['joy', 'optimism']
['disgust']
['anger', 'disgust']


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def evaluateModel(model, x_test, y_test):
    Y_pred = model.predict(x_test)

    accuracy = accuracy_score(y_test, Y_pred)
    precision = precision_score(y_test, Y_pred, average='micro')
    recall = recall_score(y_test, Y_pred, average='micro')
    roc_auc = roc_auc_score(y_test, Y_pred, average='micro')
    f1 = f1_score(y_test, Y_pred, average='micro')

    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("ROC AUC:", roc_auc)
    print("Recall:", recall)
    print("F1-Score:", f1)
    print()

print('Training Accuracy')
evaluateModel(clf, X_train, y_train)

print('Validation Accuracy')
evaluateModel(clf, X_val, y_val)

print('Testing Accuracy')
evaluateModel(clf, X_test, y_test)

Training Accuracy
Accuracy: 0.2883884176659842
Precision: 0.9549605133267522
ROC AUC: 0.7380360555565011
Recall: 0.482240777666999
F1-Score: 0.6408578999668765

Validation Accuracy
Accuracy: 0.1523702031602709
Precision: 0.8033519553072626
ROC AUC: 0.6547563217668522
Recall: 0.3327163350300787
F1-Score: 0.4705497382198953

Testing Accuracy
Accuracy: 0.16814973918379872
Precision: 0.8029705971506517
ROC AUC: 0.6567032794243043
Recall: 0.3366374380480366
F1-Score: 0.47439111747851

